# Análisis de Similaridad en Twitter

## Cargar el archivo
Se extrajeron tweets del año 2016 en los que se menciona al BBVA de la solución GNIP

Instalamos la librería necesaria para leer archivos de Excel

In [ ]:
#pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import networkx as nx
from networkx.algorithms import community
import time

In [ ]:
df = pd.read_excel('../data/Tweets_BBVA.xlsx', index_col="id")
df.head()

Extraemos el campo "body" que contiene el tweet

In [ ]:
tweet = df["body"][1:500]
tweet.head()

#### Creamos la Matriz Documento-Término con CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vec = CountVectorizer()
X = vec.fit_transform(tweet)

In [ ]:
dtm = pd.DataFrame(X.toarray().transpose(), index=vec.get_feature_names())
dtm.columns = tweet.index
dtm.head()

#### Creamos la Matriz Documento-Término con TfidfVectorizer

Es otra opción, considerando la Frecuencia Inversa de los Términos

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
doc_vec = vectorizer.fit_transform(tweet)

In [ ]:
dtm2 = pd.DataFrame(doc_vec.toarray().transpose(),
                   index=vectorizer.get_feature_names())
dtm2.columns = tweet.index
dtm2.head()

Calculamos la matriz de correlaciones

In [ ]:
matcor = dtm.corr()
matcor.head()

Transformamos la matriz en un DataFrame de input para el Grafo

In [ ]:
cordf = pd.DataFrame()
cordf = pd.DataFrame(columns = ['inicio', 'fin', 'peso'])
for i in matcor.index:
    for j in matcor.index:
        if i<j:
            try:
                w=matcor.loc[i,j]
                cordf = cordf.append({'inicio' : i, 'fin' : j, 'peso' : w}, ignore_index = True)
            except Exception:
                pass

Filtramos las correlaciones bajas y monstramos las más altas

In [ ]:
cordf = cordf[cordf['peso']>.4]
cordf.sort_values('peso', ascending=False).head(10)

Los tweets que tienen correlación 1, tienen exactamente el mismo contenido

In [ ]:
tweet[tweet.index==44670]

In [ ]:
tweet[tweet.index==44681]

Creamos el Grafo de Relaciones entre Tweets, para identificar visualmente los que sean similares

In [ ]:
G = nx.from_pandas_edgelist(cordf, source = 'inicio', target = 'fin', edge_attr='peso')
print(nx.info(G))

Crear la función top_nodes que mostrará los valores más altos de un diccionario

In [ ]:
def get_top_nodes(cdict, num=5):
    top_nodes ={}
    for i in range(num):
        top_nodes =dict(
            sorted(cdict.items(), key=lambda x: x[1], reverse=True)[:num]
            )
        return top_nodes

#### Visualización de Similaridad

Guardar el grado de cada nodo en un diccionario

In [ ]:
gdeg=G.degree()
get_top_nodes(dict(gdeg))

Ahora Visualizamos los Tweets, agrupados por similaridad

In [ ]:
plt.figure(figsize=(80,45)) 
pos=nx.spring_layout(G)
edges = G.edges()
weights = [G[u][v]['peso'] for u,v in edges]
nx.draw_networkx(G, width=weights, pos=pos, node_size=[val*10 for(node,val)in gdeg])
plt.show()

Elaborado por Luis Cajachahua bajo licencia MIT (2021)